In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Aufrufen des gesamten abgespeicherten Files
ds=xr.open_dataset("C:/Users/Aspire_NB/Desktop/era5_data/small_era5_daily.nc")
print(ds)

<xarray.Dataset>
Dimensions:    (latitude: 3, longitude: 9, time: 14610)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2018-12-31
  * longitude  (longitude) float32 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0 17.0
  * latitude   (latitude) float32 46.0 47.0 48.0
Data variables:
    r          (time, latitude, longitude) float32 ...
    q          (time, latitude, longitude) float32 ...
    msl        (time, latitude, longitude) float32 ...


In [3]:
#gleitendes Mittel über 21 Tage: 1 Fenster in der Mitte, 10 Tage davor, 10 Tage danach (=21)
ds_mean=ds.rolling(time=21,center=True).mean()

#Berechnung der Klimatologien
climatology=ds_mean.groupby("time.dayofyear").mean('time')

In [4]:
climatology

<xarray.Dataset>
Dimensions:    (dayofyear: 366, latitude: 3, longitude: 9)
Coordinates:
  * longitude  (longitude) float32 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0 17.0
  * latitude   (latitude) float32 46.0 47.0 48.0
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366
Data variables:
    r          (dayofyear, latitude, longitude) float32 49.981453 ... 52.402916
    q          (dayofyear, latitude, longitude) float32 0.001514967 ... 0.0012816202
    msl        (dayofyear, latitude, longitude) float32 101939.26 ... 102305.016

In [12]:
# Erstellen des rolling windows für die +- 21 Tage (neue Dimension)
data_std=ds.rolling(time=21,center=True).construct("window_dim")

In [13]:
#Berechnen der Standardabweichungen für alle day of years (366)
std_dayofyear=data_std.groupby("time.dayofyear").std()

C:\Users\Aspire_NB\Anaconda3\envs\Klimaenvironment\lib\site-packages\xarray\core\groupby.py:758: FutureWarning: Default reduction dimension will be changed to the grouped dimension in a future version of xarray. To silence this warning, pass dim=xarray.ALL_DIMS explicitly.
  allow_lazy=True, **kwargs)


In [14]:
std_dayofyear

<xarray.Dataset>
Dimensions:    (dayofyear: 366)
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366
Data variables:
    r          (dayofyear) float32 24.812124 24.782173 ... 24.705183 25.596664
    q          (dayofyear) float32 0.00079775834 0.0007875421 ... 0.0007700822
    msl        (dayofyear) float32 1049.0269 1042.7858 ... 1047.4121 1046.5144

In [15]:
#Anomalieberechnung und Normieren dieser
anomalies=(ds.groupby("time.dayofyear")-climatology)/std_dayofyear

In [16]:
anomalies

<xarray.Dataset>
Dimensions:    (dayofyear: 366, latitude: 3, longitude: 9, time: 14610)
Coordinates:
  * longitude  (longitude) float32 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0 17.0
  * latitude   (latitude) float32 46.0 47.0 48.0
  * time       (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2018-12-31
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366
Data variables:
    r          (time, latitude, longitude, dayofyear) float32 -0.7191197 ... -0.63973624
    q          (time, latitude, longitude, dayofyear) float32 -0.86109823 ... -1.0861742
    msl        (time, latitude, longitude, dayofyear) float32 -2.12715 ... 0.91081375